In [1]:
import sys
sys.path.append('../')

In [2]:
from scripts.utils import data_path
from scripts.cross_match_scripts import my_scaler_forward
import pandas as pd
from tqdm import tqdm
pd.set_option('display.max_columns', 500)
import seaborn as sns
import numpy as np
%matplotlib inline

matplotlib settings set


matplotlib settings set


# Catalogs loading and preprocessing

In [3]:
photo_cat = pd.read_pickle(data_path+'csc_allsky_desi_r30_gaia_dered_training.gz_pkl', compression='gzip')
x_ray_flux_lim = 3e-15
photo_cat = photo_cat.query('(csc_flux_05_2 > @x_ray_flux_lim) | is_counterpart == False')
photo_cat = photo_cat.replace([np.inf, -np.inf], np.nan)
photo_cat.sample(5)

,csc_name,csc_ra,csc_dec,csc_flux_05_2,csc_radec_err,csc_r_98,release,objid,brickid,ra,dec,brick_primary,flux_g,flux_r,flux_z,flux_w1,flux_w2,flux_w3,flux_w4,flux_ivar_g,flux_ivar_r,flux_ivar_z,flux_ivar_w1,flux_ivar_w2,flux_ivar_w3,flux_ivar_w4,dered_mag_g,dered_mag_r,dered_mag_z,dered_mag_w1,dered_mag_w2,dered_mag_w3,dered_mag_w4,snr_g,snr_r,snr_z,snr_w1,snr_w2,snr_w3,snr_w4,parallax,parallax_ivar,pmra,pmra_ivar,pmdec,pmdec_ivar,dist_arcsec,desi_id,all_mag_g,rel_mag_g,rel_dered_mag_g,all_mag_r,rel_mag_r,rel_dered_mag_r,all_mag_z,rel_mag_z,rel_dered_mag_z,all_mag_w1,rel_mag_w1,rel_dered_mag_w1,vega_mag_w1,all_mag_w2,rel_mag_w2,rel_dered_mag_w2,vega_mag_w2,all_mag_w3,rel_mag_w3,rel_dered_mag_w3,vega_mag_w3,all_mag_w4,rel_mag_w4,rel_dered_mag_w4,vega_mag_w4,rel_desi_flux_corr_z,rel_dered_g_r,rel_dered_g_z,rel_dered_r_z,rel_dered_z_w1,rel_dered_r_w2,rel_dered_w1_w2,rel_dered_z_w3,rel_dered_r_w4,rel_dered_w3_w4,r_false_003_dens_10_30,num_srcs_sep_less_r_false,is_counterpart
1555420,2CXO J222340.5-020242,335.919015,-2.045142,NaN,0.771953,NaN,9010,4646,320191,335.913605,-2.049020,1,0.125729,0.262486,0.963103,3.067882,6.670373,-100.591805,1085.048500,2808.00700,1461.30690,232.295430,2.825732,0.600912,0.000753,0.000010,24.472310,23.764227,22.435654,21.266924,20.429811,NaN,14.910587,6.662435,10.034048,14.678880,5.157083,5.170775,-2.761176,3.485655,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,23.950156,9010_320191_4646,24.751411,24.751411,24.472310,23.952235,23.952235,23.764227,22.540818,22.540818,22.435654,21.282903,21.282903,21.266924,18.583903,20.439625,20.439625,20.429811,17.100625,NaN,NaN,NaN,NaN,14.911377,NaN,NaN,NaN,1.771435e-15,0.708083,2.036656,1.328573,1.168730,3.334416,0.837113,NaN,NaN,NaN,1.274555,1,False
256956,2CXO J021833.2-051152,34.638542,-5.197831,NaN,0.582808,NaN,9010,1672,300292,34.644901,-5.202117,1,0.102720,0.249438,0.476178,1.188940,1.498015,-54.271570,28.901766,2678.46040,1540.65040,148.449940,3.388954,0.722151,0.001454,0.000019,24.903637,23.962313,23.280249,22.308252,22.058846,NaN,18.847498,5.316175,9.790712,5.801757,2.188733,1.273005,-2.069327,0.126412,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,27.525814,9010_300292_1672,24.970862,24.970862,24.903637,24.007593,24.007593,23.962313,23.305577,23.305577,23.280249,22.312100,NaN,NaN,NaN,22.061210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.847689,NaN,NaN,NaN,8.137431e-16,0.941324,1.623388,0.682064,NaN,NaN,NaN,NaN,NaN,NaN,0.949997,1,False
524107,2CXO J052457.0-471529,81.237558,-47.258066,5.488090e-15,1.754758,4.908322,9010,1738,88059,81.237625,-47.257851,1,25.956390,100.260060,513.187900,372.067020,238.523090,60.994890,-547.813660,609.57776,227.00833,29.411282,0.990956,0.556104,0.003426,0.000040,18.854324,17.423038,15.682837,16.067146,16.552303,18.035940,NaN,640.853640,1510.597900,2783.129400,370.380650,177.872390,3.570377,-3.451222,2.149493,55.083540,-1.206697,14.939035,10.553545,16.074358,0.790022,9010_88059_1738,18.964389,18.964389,18.854324,17.497180,17.497180,17.423038,15.724309,15.724309,15.682837,16.073447,16.073447,16.067146,13.374447,16.556174,16.556174,16.552303,13.217174,18.036766,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.901273e-13,1.431286,3.171487,1.740201,-0.384309,0.870735,-0.485157,NaN,NaN,NaN,0.872628,1,True
141360,2CXO J010411.0-215325,16.046147,-21.890403,NaN,0.444554,NaN,9010,7074,206713,16.053308,-21.890476,1,0.053355,0.091757,0.114375,-0.568482,1.937505,7.093524,-1362.549800,3839.66360,2584.57030,276.512300,3.699013,0.749773,0.000920,0.000011,25.617357,25.049816,24.829790,NaN,21.779617,20.372360,NaN,3.306147,4.664800,1.901907,-1.093351,1.677675,0.215177,-4.599502,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,23.923249,9010_206713_7074,25.682062,NaN,NaN,25.093402,25.093402,25.049816,24.854172,NaN,NaN,NaN,NaN,NaN,NaN,21.781893,NaN,NaN,NaN,20.372845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.780502,1,False
1418526,2CXO J171337.2+531602,258.405148,53.267285,NaN,1.604168,NaN,9011,2540,596207,258.411367,53.262537,1,4.763

In [4]:
print('fraction of missed measurements per is_counterpart per filter, in per cent:')
photo_cat.groupby('is_counterpart')[['rel_mag_g',	'rel_mag_r',	'rel_mag_z',	'rel_mag_w1',	'rel_mag_w2',	'rel_mag_w3',	'rel_mag_w4']].agg(lambda x: np.round(100*np.mean(np.
isnan(x)),1))

fraction of missed measurements per is_counterpart per filter, in per cent:


,rel_mag_g,rel_mag_r,rel_mag_z,rel_mag_w1,rel_mag_w2,rel_mag_w3,rel_mag_w4
is_counterpart,,,,,,,
False,21.1,14.0,24.4,59.5,76.2,95.6,85.8
True,7.4,4.5,2.0,5.0,9.2,70.7,79.4


In [5]:
features_cols = ['rel_dered_mag_g','rel_dered_mag_r','rel_dered_mag_z'] 
features_cols += ['rel_dered_g_r', 'rel_dered_r_z', 'rel_dered_g_z']


features_cols += ['rel_dered_mag_w1', 'rel_dered_mag_w2']
features_cols += ['rel_dered_z_w1', 'rel_dered_r_w2', 'rel_dered_w1_w2']

features_cols += ['rel_dered_mag_w3', 'rel_dered_mag_w4']
features_cols += ['rel_dered_z_w3', 'rel_dered_r_w4', 'rel_dered_w3_w4']


target_col = ['is_counterpart']

photo_cat = photo_cat[features_cols+target_col+['csc_flux_05_2']]

#photo_cat= photo_cat.dropna(subset = features_cols, how = 'any')

print(photo_cat.is_counterpart.value_counts())
photo_cat = photo_cat.sample(frac=1)
photo_cat.reset_index(inplace=True, drop = True)

photo_cat_scaled = my_scaler_forward(photo_cat)
photo_cat_scaled.to_pickle('./model/train_test_validation_data.pkl')
photo_cat_scaled.head(15)


False    368740
True      35806
Name: is_counterpart, dtype: int64
rel_dered_mag_g scaled by 1/35
rel_dered_mag_r scaled by 1/35
rel_dered_mag_z scaled by 1/35
rel_dered_g_r scaled by 1/10
rel_dered_r_z scaled by 1/10
rel_dered_g_z scaled by 1/10
rel_dered_mag_w1 scaled by 1/35
rel_dered_mag_w2 scaled by 1/35
rel_dered_z_w1 scaled by 1/10
rel_dered_r_w2 scaled by 1/10
rel_dered_w1_w2 scaled by 1/10
rel_dered_mag_w3 scaled by 1/35
rel_dered_mag_w4 scaled by 1/35
rel_dered_z_w3 scaled by 1/10
rel_dered_r_w4 scaled by 1/10
rel_dered_w3_w4 scaled by 1/10


,rel_dered_mag_g,rel_dered_mag_r,rel_dered_mag_z,rel_dered_g_r,rel_dered_r_z,rel_dered_g_z,rel_dered_mag_w1,rel_dered_mag_w2,rel_dered_z_w1,rel_dered_r_w2,rel_dered_w1_w2,rel_dered_mag_w3,rel_dered_mag_w4,rel_dered_z_w3,rel_dered_r_w4,rel_dered_w3_w4,is_counterpart,csc_flux_05_2
0,0.691683,0.681138,0.668626,0.036908,0.043792,0.080699,NaN,NaN,NaN,NaN,NaN,NaN,0.375012,NaN,1.071440,NaN,False,NaN
1,NaN,0.686543,0.645466,NaN,0.143771,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.428864,NaN,0.901876,NaN,False,NaN
2,0.722051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
3,0.708018,0.697223,NaN,0.037783,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
4,0.677138,0.671650,0.649900,0.019208,0.076124,0.095332,NaN,NaN,NaN,NaN,NaN,0.481042,NaN,0.591005,NaN,NaN,False,NaN
5,0.685361,0.671447,NaN,0.048697,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.393927,NaN,0.971321,NaN,False,NaN
6,0.695812,0.662285,0.620864,0.117345,0.144973,0.262317,0.575200,0.576799,0.159824,0.299202,-0.005595,NaN,0.417976,NaN,0.855080,NaN,False,NaN
7,0.323730,0.330203,0.333110,-0.022657,-0.010172,-0.032829,0.368868,0.386736,-0.125155,-0.197865,-0.062538,0.431148,NaN,-0.343135,NaN,NaN,True,2.506598e-14
8,0.667277,0.617225,0.591876,0.175180,0.088724,0.263905,0.578999,NaN,0.045066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
9,NaN,NaN,0.646403,NaN,NaN,NaN,0.564106,0.554851,0.288042,NaN,0.032392,NaN,NaN,NaN,NaN,NaN,False,NaN
